This notebook assumes access to denoised, detrended signals and calculates the appropriate parameters for the signal.

In [2]:
%run "Data_PreProcessing.ipynb" #allowing access to the filtered database with preprocessed signals

100%|████████████████████████████████████████████████████████████████████████████████| 229/229 [00:02<00:00, 98.94it/s]


In [3]:
from scipy.signal import find_peaks #for peak finding

In [13]:
def peak_eval(signal, distance):
    peaks, _ = find_peaks(signal, distance = distance)
    peak_to_peaks = peaks[1:-1]

    # Calculate differences between adjacent peak indices
    peak_distances = np.diff(peak_to_peaks)

    # average/std distance between peaks - basically equivalent to heart rate
    peak_av = np.mean(peak_distances)
    #peak_std = np.std(peak_distances) dont think we want this?? - can check later when do feature selection
    
    #calculating amplitude of R peak
    peak_amp = signal[peak_to_peaks] - np.median(signal) #median used as baseline
    
    peak_amp_av = np.mean(peak_amp)
    #peak_amp_std = np.std(peak_amp)
    
    return peak_amp_av, peak_av


peak_amp_av = np.zeros(shape = (no_patients))
peak_av = np.zeros(shape = (no_patients))
for i, signal in enumerate(denoised_signals):
    peak_amp_av[i], peak_av[i] = peak_eval(signal, 600)


In [11]:

#create dictionary with parameters
params = {}

params['peak_amp_av'] = peak_amp_av
params['peak_to_peak_av'] = peak_av
